In [1]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [2]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [3]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
# vocab
vocab = {'<UNK>':0}

In [5]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [6]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [7]:
len(vocab)

324

In [8]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [9]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [12]:
dataset = QADataset(df, vocab)

In [13]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [14]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([215])
tensor([[  1,   2,   3, 146,  86,  19, 192, 193]]) tensor([194])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([321])
tensor([[ 1,  2,  3,  4,  5, 99]]) tensor([100])
tensor([[ 10, 308,   3, 309, 310]]) tensor([311])
tensor([[10, 75, 76]]) tensor([77])
tensor([[ 1,  2,  3, 17, 18, 19, 20, 21, 22]]) tensor([23])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[ 42, 255,   2, 256,  83, 257, 258]]) tensor([259])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]]) tensor([220])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([49])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([91])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([268])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([7])
tensor([[ 78,  79, 195,  81,  19,   3, 196, 197, 198]]) tensor([199])


In [15]:
import torch.nn as nn

In [17]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [18]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [28]:
learning_rate = 0.001
epochs = 50

In [29]:
model = SimpleRNN(len(vocab))

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 525.583734
Epoch: 2, Loss: 457.584794
Epoch: 3, Loss: 380.393313
Epoch: 4, Loss: 318.541743
Epoch: 5, Loss: 266.092507
Epoch: 6, Loss: 217.222886
Epoch: 7, Loss: 173.498182
Epoch: 8, Loss: 134.567068
Epoch: 9, Loss: 103.899513
Epoch: 10, Loss: 79.654421
Epoch: 11, Loss: 61.141799
Epoch: 12, Loss: 47.339605
Epoch: 13, Loss: 37.446979
Epoch: 14, Loss: 29.987663
Epoch: 15, Loss: 24.258517
Epoch: 16, Loss: 20.019612
Epoch: 17, Loss: 16.808464
Epoch: 18, Loss: 14.165758
Epoch: 19, Loss: 12.150719
Epoch: 20, Loss: 10.530896
Epoch: 21, Loss: 9.200548
Epoch: 22, Loss: 8.129353
Epoch: 23, Loss: 7.203950
Epoch: 24, Loss: 6.428135
Epoch: 25, Loss: 5.784600
Epoch: 26, Loss: 5.223294
Epoch: 27, Loss: 4.751729
Epoch: 28, Loss: 4.309642
Epoch: 29, Loss: 3.951429
Epoch: 30, Loss: 3.611946
Epoch: 31, Loss: 3.313356
Epoch: 32, Loss: 3.059858
Epoch: 33, Loss: 2.830900
Epoch: 34, Loss: 2.619963
Epoch: 35, Loss: 2.430014
Epoch: 36, Loss: 2.261673
Epoch: 37, Loss: 2.105443
Epoch: 38, Loss: 1

In [32]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [36]:
predict(model, "What is capital of france?")

paris


In [27]:
list(vocab.keys())[7]

'paris'